# Blue Brain BioExplorer
![](../../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer
from mediamaker import MovieMaker
import os

hostname = 'localhost:5000'
resource_folder = '../../../tests/test_files/'
    
be = BioExplorer(hostname)
core = be.core_api()
mm = MovieMaker(be)

version = be.version()
print('Version: ' + version)

sequence_name = 'high_glucose'

In [ ]:
'''Reset current scene'''
status = be.reset()

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=True,
    model_visibility_on_creation=False)

In [ ]:
cache_folder = resource_folder + 'caches/' + version + '/'

In [ ]:
cache_filename = cache_folder + sequence_name + '_scenario.bioexplorer'
print('Loading ' + cache_filename)
dummy = core.add_model(cache_filename)

In [ ]:
nb_models = len(core.scene.models)
if nb_models != 180:
    print('ERROR: Not all models have been loaded!!! Expected 180, found ' + str(nb_models))
else:
    print('All good!')

In [ ]:
'''Now we can show the models'''
status = be.set_models_visibility(True)

## Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer', head_light=False,
    samples_per_pixel=1, subsampling=1, max_accum_frames=128)
params = be.core_api().BioExplorerRendererParams()
params.exposure = 1.0
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 5000
params.shadows = 1.0
params.soft_shadows = 0.02
params.fog_start = 1000
params.fog_thickness = 300
params.max_bounces = 1
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

status = core.clear_lights()
status = core.add_light_directional(
    angularDiameter=0.5, color=[1,1,1], direction=[-0.7,-0.4,-1],
    intensity=1.0, is_visible=False
)

In [ ]:
status = be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_DIFFUSE,
    specular_exponent=50.0
)

In [ ]:
for model in core.scene.models:
    model_id = model['id']
    model_name = model['name']
    if be.NAME_COLLAGEN in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        emissions = list()
        for i in range(nb_materials):
            palette.append([1,1,1])
            emissions.append(0.2)
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
            emissions=emissions
        )        
core.scene.commit()

## Snapshots

In [ ]:
k = 4
image_size = [k*960,k*540]
image_spp = 64
image_format = 'png'
aperture_multiplier = 0.0

dof = ''
if aperture_multiplier != 0.0:
    dof = 'dof_'
    image_format='jpg'

core.set_camera(current='bio_explorer_perspective')

output_folder = resource_folder + 'frames/' + \
                version + '/' + sequence_name + '/'

output_suffix = '_' + dof + str(image_size[0]) + 'x' + \
                str(image_size[1]) + '_' + str(image_spp) + 'spp.png'

command_line='mkdir -p ' + output_folder
os.system(command_line)
command_line='ls ' + output_folder
if os.system(command_line) != 0:
    print('ERROR: Failed to create output folder')

print(output_folder)
print(output_suffix)

In [ ]:
''' cell Membrane '''
status = mm.set_camera(
    direction = [-1.0, 0.0, 0.0],
    origin = [0.0, -160, 100],
    up = [0.0, 1.0, 0.0]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'cell_membrane' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' SP-D '''
status = mm.set_camera(
    direction = [0.277, -0.049, -0.959],
    origin = [38.749, 35.228, 5.536],
    up = [0.0, 1.0, 0.0]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'surfactant' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' Free virus '''
status = mm.set_camera(
    direction = [0.009, 0.055, -0.998],
    origin = [-0.832, 72.134, 387.389],
    up = [0.017, 0.998, 0.055]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'virus' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' Overview '''
status = mm.set_camera(
    direction = [0.009, 0.055, -0.998],
    origin = [0.293, 19.604, 1000],
    up = [0.017, 0.998, 0.055]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'overview' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

In [ ]:
''' ACE2 '''
status = mm.set_camera(
    direction = [-0.436, 0.035, -0.898],
    origin = [-33.619, -164.994, 276.296],
    up = [0.011, 0.999, 0.033]
)

In [ ]:
mm.create_snapshot(
    size=image_size,
    path=output_folder + 'receptor' + output_suffix,
    samples_per_pixel=image_spp, exportIntermediateFrames=False)

## Movie

In [ ]:
aperture_ratio = 1.0
cameras_key_frames = [
    { # Membrane
        'apertureRadius': aperture_ratio * 0.0,
        'focusDistance': 1.0,
        'direction': [-1.0, 0.0, 0.0],
        'origin': [150.0, -160, 100],
        'up': [0.0, 1.0, 0.0]
    }, {
        'apertureRadius': aperture_ratio * 0.0,
        'focusDistance': 0,
        'direction' : [0.0, 0.0, -1.0],
        'origin' : [-67.501, -17.451, 254.786],
        'up' : [0.0, 1.0, 0.0]
    }, { # Surfactant Head
        'apertureRadius': aperture_ratio * 0.01,
        'focusDistance': 30,
        'direction' : [0.276, -0.049, -0.959],
        'origin' : [38.749, 35.228, 5.536],
        'up' : [0.0, 1.0, 0.0]
    }, { # Virus overview
        'apertureRadius': aperture_ratio * 0.0,
        'focusDistance': 349.75,
        'direction' : [0.009, 0.055, -0.998],
        'origin' : [-0.832, 72.134, 387.389],
        'up' : [0.017, 0.998, 0.055]
    }, { # ACE2
        'apertureRadius': aperture_ratio * 0.01,
        'focusDistance': 45.31,
        'direction' : [-0.436, 0.035, -0.898],
        'origin' : [-33.619, -164.994, 276.296],
        'up' : [0.011, 0.999, 0.033]
    }, { # Membrane overview
        'apertureRadius': aperture_ratio * 0.0,
        'focusDistance': 60,
        'direction' : [0.009, 0.055, -0.998],
        'origin' : [0.293, 19.604, 1000],
        'up' : [0.017, 0.998, 0.055]
    }
]

In [ ]:
'''Double the frames to make it smoother'''
key_frames = list()
for cameras_key_frame in cameras_key_frames:
    key_frames.append(cameras_key_frame)
    key_frames.append(cameras_key_frame)

In [ ]:
def render(projection, k, samples_per_pixel, start_frame=0, end_frame=0):
    import os
    from mediamaker import MovieMaker

    nb_frames = core.get_animation_parameters()['frame_count']
    export_folder = output_folder + 'sequences/' + projection + '/'

    end_frame=nb_frames
    frame_step=1
    aperture_ratio=0.0

    output_size = list()
    if projection=='perspective':
        aperture_ratio = 1.0
        output_size = [int(k*960),int(k*540)]
        core.set_camera(current='bio_explorer_perspective')
    elif projection=='fisheye':
        output_size = [int(k*1024),int(k*1024)]
        core.set_camera(current='fisheye')
    elif projection=='panoramic':
        output_size = [int(k*1024),int(k*1024)]
        core.set_camera(current='panoramic')
    elif projection=='opendeck':
        output_size = [7*2160,3840]
        core.set_camera(current='cylindric')

    print('Rendering <' + sequence_name + '> sequence in <' + projection + '> projection ' + str(output_size))
    print(str(samples_per_pixel) + ' samples per pixel')
    print(export_folder)

    command_line='mkdir -p ' + export_folder
    os.system(command_line)
    command_line='ls ' + export_folder
    if os.system(command_line) != 0:
        print('ERROR: Failed to create output folder')
        
    size = core.get_application_parameters()['viewport']
    core.set_renderer(samples_per_pixel=1, max_accum_frames=samples_per_pixel)
    core.set_application_parameters(viewport=output_size)
    core.set_application_parameters(image_stream_fps=0)

    print('exporting...')
    
    mm.export_frames(
        path=export_folder, size=output_size,
        samples_per_pixel=samples_per_pixel-1,
        start_frame=start_frame, end_frame=end_frame
    )
    
    while mm.get_export_frames_progress()['progress'] < 1.0:
        import time
        time.sleep(1)
        
    mm.cancel_frames_export()

In [ ]:
mm.build_camera_path(key_frames, 250, 150)

In [ ]:
mm.set_current_frame(2000)

In [ ]:
render('perspective', 'production', 4, 64)

## Fields

In [ ]:
planes = core.get_clip_planes()
if planes:
    plane_ids = list()
    for plane in planes:
        plane_ids.append(plane['id'])
    core.remove_clip_planes(plane_ids)
    
core.add_clip_plane([0,0,1,-220])
core.add_clip_plane([0,0,-1,300])
core.add_clip_plane([1,0,0,120])
core.add_clip_plane([-1,0,0,0])
core.add_clip_plane([0,1,0,200])
core.add_clip_plane([0,-1,0,-130])

In [ ]:
status = be.build_fields(voxel_size=0.25)
fields_model_id = core.scene.models[len(core.scene.models)-1]['id']

In [ ]:
image_size = [3840, 2160]
image_samples_per_pixel = 1

output_folder = '/tmp/'
colormap_folder = resource_folder + 'colormap/'

In [ ]:
core.set_renderer(
    current='bio_explorer_fields',
    samples_per_pixel=1, subsampling=8, max_accum_frames=image_samples_per_pixel)

params = core.BioExplorerFieldsRendererParams()
params.cutoff = 5000
params.exposure = 3.0
params.alpha_correction = 0.1
params.nb_ray_steps = 16
params.nb_ray_refinement_steps = image_samples_per_pixel
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

### Transfer function

In [ ]:
from ipyTransferFunction import TransferFunctionEditor
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

def update_palette(tf):
    intensity = 1
    btf = core.get_model_transfer_function(id=fields_model_id)
    colors = list()
    points = list()
    
    nb_points = len(tf.alpha_sliders)
    step = 1.0 / float(nb_points - 1)
    for i in range(nb_points):
        color = hex_to_rgb(tf.color_pickers[i].value)
        colors.append([
            intensity * float(color[0]) / 256.0, 
            intensity * float(color[1]) / 256.0, 
            intensity * float(color[2]) / 256.0])
        points.append([i * step, tf.alpha_sliders[i].value / 255.0])
        
    btf['colormap']['name'] = 'TransferFunctionEditor'
    btf['colormap']['colors'] = colors
    btf['opacity_curve'] = points
    btf['range'] = [tf.data_range[0], tf.data_range[1]]
    core.set_model_transfer_function(id=fields_model_id, transfer_function=btf)

In [ ]:
tf = TransferFunctionEditor(
    filename=colormap_folder + 'spike_v1.1dt',
    on_change=update_palette)
tf.set_range((0, 10))

### Export field data to file

In [ ]:
be = BioExplorer(hostname)
core = be.core_api()
fields_model_id = core.scene.models[len(core.scene.models)-1]['id']
filename = cache_folder + 'spike_on_ace2.fields'
print(filename)
be.export_fields_to_file(
    model_id=fields_model_id,
    filename=filename)